In [ ]:
import yfinance

df = yfinance.download('AAPL', start='2020-01-01', end='2020-12-01')
df.to_csv('AAPL.csv')

In [ ]:
import sqlite3
connection = sqlite3.connect('app.db')

cursor = connection.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS stock (
    id iNTEGER PRIMART KEY,
    symbol TEXT NOT NULL UNIQUE,
    company TEXT NOT NULL
    )
""")

cursor.execute("""
               CREATE TABLE IF NOT EXISTS stock_price (
                    id INTEGER PRIMARY KEY,
                    stock_id INTEGER,
                    date NOT NULL,
                    open NOT NULL,
                    high NOT NULL,
                    low NOT NULL,
                    close NOT NULL,
                    adjusted_close NOT NULL,
                    volume NOT NULL,
                    FOREIGN KEY (stock_id) REFERENCES stock (id)
                )
               """
)

connection.commit()


In [ ]:
from dotenv import load_dotenv
import os
import sqlite3
import alpaca_trade_api as tradeapi

# Load environment variables from the .env file
load_dotenv()

connection = sqlite3.connect('app.db')

cursor = connection.cursor()


api = tradeapi.REST(os.getenv('ALPACA_KEY'), os.getenv('ALPACA_SECRET'), base_url='https://paper-api.alpaca.markets')
assets = api.list_assets()

for asset in assets:
    print(asset)

In [ ]:
from dotenv import load_dotenv
import os
import sqlite3
import alpaca_trade_api as tradeapi
import pandas as pd
from datetime import datetime
import time

# Load environment variables from the .env file
load_dotenv()

connection = sqlite3.connect('app.db')

# Set row factory to sqlite3.Row to access columns by name
connection.row_factory = sqlite3.Row
cursor = connection.cursor()
cursor.execute("""SELECT id, symbol, name FROM stock""")
rows = cursor.fetchall()

symbols = []
stock_dict = {}
for row in rows:
    symbol = row['symbol']
    symbols.append(symbol)
    stock_dict[symbol] = row['id']

api = tradeapi.REST(os.getenv('ALPACA_KEY'), os.getenv('ALPACA_SECRET'), base_url='https://paper-api.alpaca.markets', api_version='v2')

timeframe = tradeapi.TimeFrame.Day
start_date = pd.Timestamp('2022-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2023-02-01', tz='America/New_York').isoformat()
chunk_size = 200
for i in range(0, len(symbols), chunk_size):
        symbol_chunk = symbols[i:i+chunk_size]
        bars = api.get_bars(symbol_chunk, timeframe, start=start_date, end=end_date).df
        bars.to_sql('stock_data', con=connection, if_exists='replace', index_label='timestamp')
        print(f"Inserting records from {i} to {i + chunk_size}")
        time.sleep(3)

In [ ]:
unique_symbols = bars['symbol'].unique()
for symbols in unique_symbols:
    print(symbols)
# cursor.execute("""
#                INSERT INTO stock_price (stock_id, date, open, high, low, close, volume)
#                VALUES (?, ?, ?, ?, ?, ?, ?)
#                """, (stock_id, ))

In [ ]:
for idx, row in bars.iterrows():
    cursor.execute("""
        INSERT INTO stock_price (stock_id, date, open, high, low, close, volume)
        SELECT s.id, ?, ?, ?, ?, ?, ?
        FROM stock s
        WHERE s.symbol = ?
    """, (idx.strftime('%Y-%m-%d %H:%M:%S'), row['open'], row['high'], row['low'], row['close'], row['volume'], row['symbol']))

    # Commit the changes to the stock_price table
    connection.commit()

In [ ]:
# Prepare data for bulk insert into stock_price
records_to_insert = []
for idx, row in bars.iterrows():
    cursor.execute("SELECT id FROM stock WHERE symbol = ?", (row['symbol'],))
    stock_id = cursor.fetchone()[0]
    records_to_insert.append((stock_id, idx.strftime('%Y-%m-%d %H:%M:%S'), row['open'], row['high'], row['low'], row['close'], row['volume']))

# Bulk insert into stock_price
cursor.executemany("""
    INSERT INTO stock_price (stock_id, date, open, high, low, close, volume)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""", records_to_insert)

# Commit the changes to the stock_price table
connection.commit()